In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from math import sqrt
from sklearn.metrics import make_scorer, mean_squared_error,r2_score,accuracy_score,mean_absolute_error,explained_variance_score
from collections import Counter

from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
#from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
import warnings
warnings.simplefilter(action='ignore')

In [2]:
data = pd.read_csv('Cleaned_data.csv',low_memory=False)
data.head()

,Unnamed: 0,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,real_dispos_income_per_cap_growth,water_pipes_share,rent_price_4+room_bus,rent_price_3room_bus,rent_price_2room_bus,rent_price_1room_bus,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,month
0,753,38.0,19.0,16.0,17.0,1.0,1980.0,2.0,6.0,2.0,...,-0.025,98.8,153.86,97.07,69.15,51.77,47.19,41.19,32.69,1
1,754,56.0,56.0,2.0,9.0,1.0,1980.0,2.0,6.0,2.0,...,-0.025,98.8,153.86,97.07,69.15,51.77,47.19,41.19,32.69,1
2,755,53.0,32.0,16.0,17.0,1.0,1980.0,2.0,6.0,3.0,...,-0.025,98.8,153.86,97.07,69.15,51.77,47.19,41.19,32.69,1
3,756,72.0,38.0,29.0,44.0,1.0,1980.0,2.0,6.0,2.0,...,-0.025,98.8,153.86,97.07,69.15,51.77,47.19,41.19,32.69,1
4,757,47.0,30.0,10.0,16.0,1.0,1980.0,2.0,6.0,2.0,...,-0.025,98.8,153.86,97.07,69.15,51.77,47.19,41.19,32.69,1


In [3]:
data['product_type']

0           Investment
1           Investment
2           Investment
3        OwnerOccupier
4           Investment
             ...      
28564       Investment
28565       Investment
28566    OwnerOccupier
28567       Investment
28568       Investment
Name: product_type, Length: 28569, dtype: object

In [4]:
data.drop('Unnamed: 0',axis=1,inplace=True)

In [5]:
obj = data.select_dtypes(include=['object'])
num = data.select_dtypes(exclude=['object'])

In [6]:
for col in  obj.columns:
    print(Counter(obj[col]),'\n')

Counter({'Investment': 17728, 'OwnerOccupier': 10841}) 

Counter({'Poselenie Sosenskoe': 1756, 'Nekrasovka': 1601, 'Poselenie Vnukovskoe': 1367, 'Poselenie Moskovskij': 921, 'Poselenie Voskresenskoe': 680, 'Mitino': 671, 'Tverskoe': 669, 'Krjukovo': 508, 'Poselenie Filimonkovskoe': 496, "Mar'ino": 492, 'Juzhnoe Butovo': 434, 'Poselenie Shherbinka': 433, 'Solncevo': 414, 'Zapadnoe Degunino': 402, 'Poselenie Desjonovskoe': 357, 'Otradnoe': 339, 'Nagatinskij Zaton': 316, 'Nagornoe': 295, 'Strogino': 295, "Tekstil'shhiki": 289, 'Ljublino': 285, 'Bogorodskoe': 284, 'Izmajlovo': 284, "Gol'janovo": 283, 'Severnoe Tushino': 276, 'Chertanovo Juzhnoe': 262, 'Birjulevo Vostochnoe': 260, 'Vyhino-Zhulebino': 257, 'Zjuzino': 251, 'Horoshevo-Mnevniki': 251, 'Ochakovo-Matveevskoe': 240, 'Perovo': 231, 'Jasenevo': 230, 'Bibirevo': 223, 'Kosino-Uhtomskoe': 221, 'Golovinskoe': 218, 'Ramenki': 218, "Kon'kovo": 214, 'Caricyno': 210, "Kuz'minki": 206, 'Akademicheskoe': 206, 'Veshnjaki': 203, 'Orehovo-Boriso

In [7]:
obj.drop('sub_area',axis=1,inplace=True)
obj_col = obj.columns
obj_col

Index(['product_type', 'culture_objects_top_25', 'thermal_power_plant_raion',
       'incineration_raion', 'oil_chemistry_raion', 'radiation_raion',
       'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion',
       'detention_facility_raion', 'water_1line', 'big_road1_1line',
       'railroad_1line', 'ecology'],
      dtype='object')

In [8]:
lt = ['product_type', 'culture_objects_top_25', 'thermal_power_plant_raion','incineration_raion','oil_chemistry_raion', 'radiation_raion','railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion','detention_facility_raion', 'water_1line', 'big_road1_1line','railroad_1line', 'ecology']

In [9]:
data['product_type']

0           Investment
1           Investment
2           Investment
3        OwnerOccupier
4           Investment
             ...      
28564       Investment
28565       Investment
28566    OwnerOccupier
28567       Investment
28568       Investment
Name: product_type, Length: 28569, dtype: object

In [10]:
dub = pd.get_dummies(obj)
data = pd.concat([data,dub],axis=1)

In [11]:
data['product_type']

0           Investment
1           Investment
2           Investment
3        OwnerOccupier
4           Investment
             ...      
28564       Investment
28565       Investment
28566    OwnerOccupier
28567       Investment
28568       Investment
Name: product_type, Length: 28569, dtype: object

In [12]:
data.drop(lt,axis=1,inplace=True)

In [13]:
data

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,sub_area,...,water_1line_yes,big_road1_1line_no,big_road1_1line_yes,railroad_1line_no,railroad_1line_yes,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory
0,38.0,19.0,16.0,17.0,1.0,1980.0,2.0,6.0,2.0,Alekseevskoe,...,0,1,0,1,0,0,0,0,1,0
1,56.0,56.0,2.0,9.0,1.0,1980.0,2.0,6.0,2.0,Timirjazevskoe,...,0,1,0,1,0,0,0,0,1,0
2,53.0,32.0,16.0,17.0,1.0,1980.0,2.0,6.0,3.0,Jasenevo,...,0,0,1,1,0,0,1,0,0,0
3,72.0,38.0,29.0,44.0,1.0,1980.0,2.0,6.0,2.0,Bogorodskoe,...,0,1,0,1,0,0,0,0,1,0
4,47.0,30.0,10.0,16.0,1.0,1980.0,2.0,6.0,2.0,Altuf'evskoe,...,0,1,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28564,44.0,27.0,7.0,9.0,1.0,1975.0,2.0,6.0,3.0,Otradnoe,...,0,1,0,1,0,0,1,0,0,0
28565,86.0,59.0,3.0,9.0,2.0,1935.0,4.0,10.0,3.0,Tverskoe,...,0,1,0,1,0,0,0,0,1,0
28566,45.0,29.0,10.0,20.0,1.0,1980.0,1.0,1.0,1.0,Poselenie Vnukovskoe,...,0,1,0,1,0,0,0,1,0,0
28567,64.0,32.0,5.0,15.0,1.0,2003.0,2.0,11.0,2.0,Obruchevskoe,...,0,1,0,1,0,0,0,0,0,1


In [14]:
nu_lt = num.columns
nu_lt

Index(['full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year',
       'num_room', 'kitch_sq', 'state', 'area_m',
       ...
       'real_dispos_income_per_cap_growth', 'water_pipes_share',
       'rent_price_4+room_bus', 'rent_price_3room_bus', 'rent_price_2room_bus',
       'rent_price_1room_bus', 'rent_price_3room_eco', 'rent_price_2room_eco',
       'rent_price_1room_eco', 'month'],
      dtype='object', length=156)

In [15]:
for i in nu_lt:
    print(Counter(data[i]),i,'\n')

Counter({38.0: 1805, 37.0: 1155, 39.0: 955, 44.0: 930, 45.0: 889, 40.0: 831, 32.0: 770, 43.0: 770, 41.0: 725, 51.0: 713, 42.0: 660, 52.0: 621, 53.0: 617, 63.0: 603, 60.0: 601, 64.0: 594, 35.0: 565, 58.0: 548, 59.0: 547, 34.0: 545, 61.0: 532, 54.0: 510, 36.0: 500, 31.0: 473, 62.0: 467, 46.0: 458, 79.0: 449, 57.0: 441, 56.0: 429, 74.0: 429, 77.0: 422, 55.0: 382, 76.0: 364, 50.0: 348, 30.0: 347, 48.0: 336, 47.0: 324, 75.0: 301, 78.0: 300, 33.0: 292, 65.0: 287, 82.0: 242, 29.0: 239, 73.0: 238, 49.0: 230, 72.0: 198, 84.0: 183, 67.0: 178, 80.0: 166, 71.0: 152, 68.0: 150, 66.0: 139, 69.0: 138, 70.0: 132, 83.0: 131, 81.0: 131, 85.0: 123, 27.0: 93, 28.0: 84, 86.0: 68, 102.0: 64, 89.0: 61, 87.0: 59, 90.0: 59, 26.0: 56, 100.0: 53, 96.0: 51, 88.0: 47, 95.0: 44, 101.0: 42, 92.0: 39, 94.0: 39, 99.0: 38, 104.0: 37, 103.0: 36, 98.0: 35, 93.0: 35, 91.0: 32, 97.0: 30, 134.0: 28, 19.0: 26, 112.0: 26, 106.0: 25, 117.0: 25, 107.0: 24, 18.0: 23, 25.0: 20, 120.0: 19, 110.0: 18, 20.0: 17, 109.0: 17, 116.0: 17

Counter({13890: 1756, 247469: 1601, 17790: 1367, 7341: 921, 9553: 680, 21155: 671, 116742: 669, 221709: 508, 2942: 496, 165727: 492, 102618: 434, 39873: 433, 125111: 414, 87713: 402, 21819: 357, 61396: 339, 76284: 316, 115352: 295, 156377: 295, 101982: 289, 89971: 285, 1452550: 284, 122862: 284, 12327: 283, 53786: 276, 112221: 262, 1716730: 260, 1318695: 257, 76670: 251, 155427: 251, 111023: 240, 36154: 231, 178264: 230, 86206: 223, 102828: 221, 94564: 218, 61039: 218, 123000: 214, 145576: 210, 31167: 206, 1362363: 206, 104410: 203, 129207: 195, 125354: 193, 72131: 192, 102590: 192, 78810: 192, 142462: 190, 111874: 190, 77878: 190, 157010: 186, 85083: 183, 43795: 182, 57999: 181, 85721: 177, 78507: 177, 83844: 176, 79576: 172, 83369: 171, 81887: 167, 123280: 165, 78616: 165, 105663: 164, 100846: 164, 90114: 159, 41504: 157, 60315: 156, 130396: 155, 57107: 152, 139322: 152, 76156: 151, 178473: 150, 8384: 147, 64317: 144, 153248: 143, 112804: 142, 57405: 142, 132349: 141, 27992: 134, 143

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Counter({2.493235501: 973, 0.269716418: 596, 1.869427067: 580, 1.445152279: 363, 1.884938815: 318, 5.980372449: 284, 0.989110339: 273, 0.822323916: 263, 0.252644637: 254, 0.341012026: 231, 0.6800168259999999: 218, 2.9139772180000003: 215, 1.080226534: 200, 0.5640692070000001: 189, 1.519553042: 184, 3.897271344: 173, 1.992371265: 172, 5.54192364: 146, 2.757150079: 143, 7.354661892: 142, 2.97212885: 141, 2.067629781: 138, 0.0: 133, 0.844726744: 132, 0.530377329: 116, 4.8727236860000005: 113, 2.085627198: 106, 1.4066758119999998: 88, 0.355630489: 85, 3.0044257880000003: 80, 2.086924697: 77, 0.731839713: 72, 1.752289935: 71, 3.13184755: 71, 0.124655337: 69, 3.373902554: 67, 3.5947396030000003: 62, 3.091088274: 57, 1.274855096: 55, 0.5252466370000001: 54, 1.411454942: 54, 0.978219737: 53, 3.960621851: 53, 1.823440184: 50, 0.193942653: 47, 0.147208753: 47, 0.976654941: 46, 1.454563775: 46, 0.13925736: 44, 2.152551407: 42, 0.120689469: 40, 1.359893297: 40, 1.503480004: 39, 1.361867402: 38, 2.

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

Counter({0.0: 1026, 21.22: 991, 7.51: 597, 32.38: 581, 31.84: 369, 34.64: 318, 14.96: 276, 0.67: 271, 10.96: 269, 21.7: 236, 13.93: 219, 23.34: 203, 15.52: 190, 36.66: 184, 93.45: 173, 43.01: 172, 11.57: 150, 7.26: 145, 40.68: 142, 45.25: 138, 28.97: 133, 3.02: 117, 40.08: 113, 40.95: 107, 10.43: 100, 23.74: 92, 37.18: 88, 6.45: 80, 32.62: 77, 18.59: 75, 12.73: 73, 46.55: 73, 28.56: 70, 0.48: 64, 7.43: 62, 7.3: 62, 13.63: 61, 10.33: 60, 15.71: 57, 15.51: 57, 15.38: 55, 10.5: 55, 29.57: 55, 20.21: 52, 1.68: 50, 1.1: 48, 17.53: 45, 15.49: 45, 7.75: 45, 12.35: 44, 36.05: 44, 26.71: 42, 8.86: 42, 3.54: 41, 7.48: 41, 3.39: 41, 20.25: 41, 29.51: 41, 15.75: 40, 31.39: 40, 9.99: 39, 1.83: 38, 13.4: 38, 5.54: 38, 48.98: 38, 22.47: 37, 5.13: 37, 10.51: 37, 37.25: 37, 10.3: 36, 3.59: 35, 4.94: 35, 29.65: 35, 65.34: 35, 6.86: 34, 1.78: 34, 16.11: 34, 3.8: 34, 8.15: 34, 7.45: 33, 17.37: 33, 8.99: 32, 19.19: 32, 15.73: 32, 7.74: 31, 5.57: 31, 3.23: 31, 44.84: 31, 23.2: 30, 5.17: 30, 4.97: 30, 4.98: 

Counter({500.0: 2267, 719.2628363598186: 1706, 650.0: 1297, 1000.0: 1289, 891.63: 596, 700.0: 569, 716.67: 465, 833.33: 459, 660.0: 456, 300.0: 346, 600.0: 339, 733.33: 300, 750.0: 294, 480.0: 273, 566.67: 256, 757.14: 250, 1833.33: 200, 525.0: 180, 666.67: 145, 633.33: 135, 776.92: 124, 810.0: 122, 620.0: 115, 615.38: 111, 533.33: 107, 1250.0: 103, 550.0: 101, 616.67: 100, 775.0: 98, 637.5: 94, 736.36: 93, 560.0: 92, 966.67: 92, 757.69: 89, 520.0: 87, 748.0: 83, 1500.0: 83, 642.86: 80, 800.0: 80, 680.0: 79, 712.5: 79, 485.71: 78, 540.0: 68, 709.09: 67, 988.89: 67, 627.27: 66, 657.14: 65, 522.22: 65, 516.67: 63, 685.71: 62, 653.85: 61, 760.0: 61, 462.5: 58, 640.0: 57, 766.67: 56, 676.83: 56, 675.0: 56, 635.29: 55, 671.43: 54, 662.5: 54, 987.5: 54, 625.0: 53, 677.78: 53, 827.27: 52, 714.29: 51, 662.96: 50, 933.33: 50, 488.89: 49, 585.71: 49, 683.33: 48, 614.29: 48, 575.0: 48, 670.59: 48, 1026.32: 47, 606.25: 46, 590.0: 43, 900.0: 42, 646.15: 41, 665.79: 41, 617.24: 40, 570.0: 40, 646.67


Counter({662.5: 1008, 700.0: 757, 747.37: 739, 661.54: 644, 507.14: 637, 880.53: 596, 783.78: 363, 1250.0: 315, 1114.29: 297, 764.3709762206872: 292, 715.04: 275, 1176.92: 273, 1191.67: 254, 540.0: 236, 889.39: 232, 771.43: 226, 850.0: 223, 821.74: 197, 600.0: 182, 776.47: 169, 815.38: 166, 760.0: 158, 753.85: 146, 870.59: 142, 730.77: 134, 672.62: 118, 733.33: 114, 1030.0: 113, 758.82: 109, 795.24: 103, 1285.71: 86, 905.95: 85, 656.0: 79, 775.0: 79, 1000.0: 79, 868.13: 79, 761.11: 79, 650.0: 73, 690.27: 72, 1108.33: 71, 781.82: 68, 836.36: 62, 668.0: 61, 757.89: 58, 795.83: 57, 737.77: 54, 705.97: 49, 740.7: 46, 750.0: 44, 1128.57: 44, 712.5: 42, 675.0: 42, 1287.5: 41, 798.1: 40, 722.7: 40, 691.67: 39, 689.47: 38, 970.75: 38, 816.67: 37, 725.0: 37, 704.76: 37, 686.49: 37, 755.99: 37, 954.37: 36, 795.45: 36, 869.43: 36, 1416.67: 35, 653.85: 35, 1875.0: 35, 666.67: 34, 706.0: 34, 753.33: 34, 676.0: 33, 1130.0: 32, 780.0: 32, 709.09: 31, 930.94: 31, 838.89: 30, 748.41: 30, 671.25: 30, 9

Counter({243.21: 221, 233.92: 216, 241.92: 199, 244.52: 178, 247.18: 175, 242.52: 154, 242.6: 154, 234.48: 154, 242.35: 145, 242.15: 142, 248.02: 130, 248.14: 129, 245.13: 127, 244.55: 125, 241.86: 124, 245.45: 121, 244.36: 119, 247.63: 119, 247.79: 119, 245.72: 118, 243.31: 114, 248.44: 113, 241.41: 112, 244.63: 111, 247.09: 111, 245.17: 110, 236.5: 110, 244.02: 109, 243.6: 109, 242.22: 104, 244.79: 103, 248.01: 102, 230.15: 100, 246.07: 99, 241.97: 97, 245.81: 97, 243.04: 97, 243.91: 94, 248.38: 94, 248.97: 92, 247.19: 91, 244.82: 90, 244.49: 89, 244.66: 89, 243.94: 88, 211.56: 87, 233.41: 87, 244.77: 87, 246.56: 86, 242.77: 86, 245.75: 85, 247.74: 85, 248.28: 85, 245.56: 84, 248.42: 83, 243.42: 82, 246.77: 81, 245.6: 81, 236.95: 80, 248.61: 79, 242.89: 79, 245.33: 77, 245.73: 76, 248.43: 75, 247.05: 75, 241.84: 74, 229.24: 73, 242.1: 73, 246.95: 72, 245.95: 72, 247.43: 71, 245.41: 70, 248.82: 70, 248.86: 70, 248.87: 70, 243.09: 69, 244.81: 69, 248.16: 69, 228.86: 68, 242.58: 68, 245

Counter({61.88: 1291, 59.03: 1232, 59.94: 1209, 59.0: 1195, 68.9: 1193, 55.3: 1187, 51.77: 1141, 59.19: 1128, 58.75: 1096, 65.13: 1041, 72.02: 936, 60.06: 926, 55.99: 877, 57.26: 766, 56.6: 750, 60.43: 723, 52.53: 722, 50.64: 720, 52.0: 666, 54.5: 630, 59.91: 615, 52.62: 578, 51.01: 574, 44.49: 516, 45.17: 511, 49.86: 493, 49.1: 485, 52.41: 464, 51.88: 463, 51.74: 457, 55.22: 440, 53.15: 397, 57.58: 386, 61.92: 349, 52.57: 334, 50.93: 325, 51.51: 315, 52.59: 310, 51.86: 310, 50.74: 303, 56.45: 110, 52.36: 90, 50.5: 90, 54.42: 79, 51.19: 74, 53.17: 72}) rent_price_1room_bus 

Counter({53.21: 1291, 50.24: 1232, 49.89: 1209, 48.7: 1195, 52.82: 1193, 49.75: 1187, 49.32: 1128, 47.04: 1096, 51.41: 1041, 52.81: 936, 49.24: 926, 49.29: 877, 48.28: 766, 48.5: 750, 50.93: 723, 47.09: 720, 47.35: 689, 47.56: 666, 48.22: 630, 49.12: 615, 46.56: 578, 48.08: 574, 50.36: 516, 50.41: 511, 50.42: 493, 46.94: 485, 45.2: 464, 47.3: 463, 48.97: 457, 47.19: 452, 52.64: 440, 45.31: 397, 46.29: 386, 43.42: 3

In [16]:
enc = LabelEncoder()
sc = StandardScaler()
data['sub_area'] = enc.fit_transform(data['sub_area'])


In [17]:
X = data.loc[:,data.columns != 'price_doc'].values
y = data.loc[:,['price_doc']].values


In [18]:
X = sc.fit_transform(X)
y = sc.fit_transform(y)
y = y.ravel()

In [19]:
data.price_doc

0         8000000
1         1000000
2         7900000
3         8965200
4         7000000
           ...   
28564     7400000
28565    25000000
28566     6970959
28567    13500000
28568     5600000
Name: price_doc, Length: 28569, dtype: int64

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=None)

In [21]:
def error(y_test,y_predict):
    print('mean absolute error(MAE):',mean_absolute_error(y_test, y_predict))
    print('mean squared error(MSE):',mean_squared_error(y_test, y_predict))
    print('root mean squared error(RMSE):',sqrt(mean_squared_error(y_test, y_predict)))
    print('R2:',r2_score(y_test, y_predict))
    print('explained variance score',explained_variance_score(y_test, y_predict))

In [22]:
ml = GradientBoostingRegressor()
ml.fit(X_train,y_train)
y_predict = ml.predict(X_test)
r2score = r2_score(y_test,y_predict)
error(y_test,y_predict)

mean absolute error(MAE): 0.32168758075644033
mean squared error(MSE): 0.4053446790684574
root mean squared error(RMSE): 0.636666850926336
R2: 0.6316875041286523
explained variance score 0.6316976845454699


In [ ]:
random_grid = {'n_estimators': [100,150,200,250,300],
               'learning_rate': [0.1,0.01],
               #'max_depth': [150,250],
               'min_samples_split': [7,8,],
               'min_samples_leaf': [1, 2,3,],
               
              }
ml1 = GradientBoostingRegressor()
gbr = GridSearchCV(estimator = ml1, param_grid = random_grid,cv = 5, verbose=2, n_jobs = 1)
gbr.fit(X_train,y_train)

In [55]:
gbr.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=3, min_samples_split=8,
                          min_weight_fraction_leaf=0.0, n_estimators=300,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [58]:
y_predict = gbr.best_estimator_.predict(X_test)
error(y_test,y_predict)
gbr.best_params_

mean absolute error(MAE): 0.3025941422057032
mean squared error(MSE): 0.3143124971551361
root mean squared error(RMSE): 0.5606357972473182
R2: 0.6599256136232688
explained variance score 0.660044755750695


{'learning_rate': 0.1,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 300}

In [34]:
ml = GradientBoostingRegressor(alpha=0.8, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=4,
                          max_features='auto',min_samples_leaf=3, 
                          min_samples_split=8,n_estimators=310,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, 
                          )
ml.fit(X_train,y_train)
y_predict = ml.predict(X_test)
error(y_test,y_predict)

mean absolute error(MAE): 0.29822770368617063
mean squared error(MSE): 0.3780599900743698
root mean squared error(RMSE): 0.6148658309536885
R2: 0.6564794711172919
explained variance score 0.6564834500298211


In [24]:
ml.feature_importances_.shape

(187,)

In [25]:
X.shape

(28569, 187)

In [26]:
table = data.columns.tolist()
table.remove('price_doc')
tablefea = ml.feature_importances_.tolist()

In [27]:
imp_fea_dict = {i:j for i,j in zip(table,tablefea)}

In [28]:
for k in imp_fea_dict.keys():
    print(imp_fea_dict[k])

0.5405638353801444
0.005994286431752458
0.004858878869302733
0.00606646330784102
0.0002616445573145052
0.009496626373625097
0.006694308663771204
0.009375206538164673
0.007265564800644466
7.206164419241211e-05
0.0009498663895358931
0.00445121406007256
0.004618163106478313
0.007522686990062908
0.0026357992587663984
0.0008788177883669536
0.0003585207250732337
0.000925663536945718
8.47514896546252e-05
0.000205211977698172
0.005095309865900867
0.00012524680153755876
4.2973154372712655e-05
9.160476232920433e-05
0.005658971739675701
0.0006326653466748978
0.0003909319721691986
0.005929939226954457
0.0
0.0
0.004425037616295344
0.0025241376194527166
0.00013865288930053186
0.00014635809204044666
0.0
0.00011116137180176524
0.00016881514628862819
3.0140693342525543e-05
0.0008003237272556474
0.010716490333556209
0.009784540882536764
0.0011160822157674924
0.0007859971054949068
0.0012927946041817703
0.004769660896270495
0.0002187518567176187
0.002290299431192659
0.002122673804021487
0.0006848416375931

In [29]:
 final_imp_fea_dict = {k: v for k,v in sorted(imp_fea_dict.items(), reverse=True, key=lambda a: a[1])}

In [30]:
final_imp_fea_dict         

{'full_sq': 0.5405638353801444,
 'ttk_km': 0.1124371969674436,
 'nuclear_reactor_km': 0.032004612667911284,
 'swim_pool_km': 0.02537497152380502,
 'trc_sqm_5000': 0.018516707277682988,
 'trc_sqm_3000': 0.01640471574340392,
 'mosque_km': 0.013221263514477293,
 'ID_metro': 0.010716490333556209,
 'metro_min_avto': 0.009784540882536764,
 'build_year': 0.009496626373625097,
 'kitch_sq': 0.009375206538164673,
 'theater_km': 0.008334790955022403,
 'university_km': 0.007846798619776484,
 'indust_part': 0.007522686990062908,
 'state': 0.007265564800644466,
 'num_room': 0.006694308663771204,
 'max_floor': 0.00606646330784102,
 'life_sq': 0.005994286431752458,
 'cpi': 0.005940465607557112,
 'build_count_block': 0.005929939226954457,
 'leisure_count_500': 0.005826012506977849,
 'office_raion': 0.005658971739675701,
 'sport_objects_raion': 0.005095309865900867,
 'floor': 0.004858878869302733,
 'industrial_km': 0.004769660896270495,
 'green_zone_part': 0.004618163106478313,
 'raion_popul': 0.0044512

In [31]:
lt = [ k for k in imp_fea_dict.keys() if imp_fea_dict[k]==0]

In [32]:
len(lt)

38

In [33]:
data2 = data
data2.drop(lt,axis=1,inplace=True)
X2 = data2.loc[:,data.columns != 'price_doc'].values
y2 = data2.loc[:,['price_doc']].values
X2 = sc.fit_transform(X2)
y2 = sc.fit_transform(y2)
y2 = y2.ravel()
X2_train, X2_test, y2_train, y2_test = train_test_split(X2,y2, test_size=0.25, random_state=None)

In [48]:
ml2 = GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=4,
                          max_features='auto',min_samples_leaf=3, 
                          min_samples_split=8,n_estimators=310,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, 
                          )
ml2.fit(X2_train,y2_train)

y1_predict = ml2.predict(X2_train)
error(y2_train,y1_predict)

y2_predict = ml2.predict(X2_test)
error(y2_test,y2_predict)

mean absolute error(MAE): 0.2368347251097339
mean squared error(MSE): 0.1533277431948091
root mean squared error(RMSE): 0.39157086612107533
R2: 0.8461705122013967
explained variance score 0.8461705122013967
mean absolute error(MAE): 0.29638822163478107
mean squared error(MSE): 0.3136721871949295
root mean squared error(RMSE): 0.5600644491439619
R2: 0.6893615977654954
explained variance score 0.6894674465641685


In [49]:
ml3 = GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=4,
                          max_features='auto',min_samples_leaf=3, 
                          min_samples_split=8,n_estimators=310,
                          n_iter_no_change=None, presort='auto',
                          random_state=100, 
                          )
ml3.fit(X2_train,y2_train)

y1_predict = ml3.predict(X2_train)
error(y2_train,y1_predict)

y2_predict = ml3.predict(X2_test)
error(y2_test,y2_predict)

mean absolute error(MAE): 0.2368347251097339
mean squared error(MSE): 0.1533277431948091
root mean squared error(RMSE): 0.39157086612107533
R2: 0.8461705122013967
explained variance score 0.8461705122013967
mean absolute error(MAE): 0.2963712071231241
mean squared error(MSE): 0.3128505280143012
root mean squared error(RMSE): 0.5593304282928842
R2: 0.6901753099958792
explained variance score 0.6902758073016975


In [ ]:
R2 = 0
RMSE = 100
for i in range(2,20,1):
    for j in range(2,20,1):
        for k in range(2,20,1):
            ml4 = GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=i,
                          max_features='auto',min_samples_leaf=j, 
                          min_samples_split=k,n_estimators=310,
                          n_iter_no_change=None, presort='auto',
                          random_state=100, 
                          )
            ml4.fit(X2_train,y2_train)
            y3_predict = ml4.predict(X2_test)
            
            rmse = sqrt(mean_squared_error(y2_test,y3_predict))
            r2    = r2_score(y2_test,y3_predict)
            if (R2<r2):
                R2 = r2
                print(i,j,k,R2,rmse)
            


2 2 2 0.6775073973449719 0.5706506454772491
3 2 2 0.6818863556002766 0.5667631204910101
3 3 2 0.6880198241652474 0.5612727188687372
3 7 18 0.688420582456908 0.5609121078499192
4 3 2 0.6932074412409852 0.5565867264800731
4 3 13 0.6937677996328888 0.5560781897169765
4 4 16 0.6940498545434773 0.5558220430479986
